# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 570.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.1/872.1 kB 6.6 MB/s eta 0:00:00


In [ ]:
import cv2 as cv2
from ultralytics import YOLO

## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [ ]:
vid_path = '/content/drive/My Drive/YOLO/Datasets/VideoNum2.mp4'
video = cv2.VideoCapture(vid_path)
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
fps = video.get(cv2.CAP_PROP_FPS)

In [ ]:
print(width)
print(height)

3840
2160


In [ ]:
import cv2
from ultralytics import YOLO, solutions

# Load the YOLO model
model = YOLO("yolov8n.pt")

# Open the video file
cap = cv2.VideoCapture("/content/drive/My Drive/YOLO/Datasets/VideoNum2.mp4")
assert cap.isOpened(), "Error reading video file"

# Get video properties
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

# Define very thin region for counting
region_width = w // 2
region_height = h // 50  # Very small height, about 2% of the frame height
region_top = (h - region_height) // 2
region_bottom = region_top + region_height
middle_x = w // 2

left_region_points = [
    (0, int(region_bottom)),               # Bottom-left
    (int(middle_x), int(region_bottom)),   # Bottom-right
    (int(middle_x), int(region_top)),      # Top-right
    (0, int(region_top))                   # Top-left
]

right_region_points = [
    (int(middle_x), int(region_bottom)),   # Bottom-left
    (int(w), int(region_bottom)),          # Bottom-right
    (int(w), int(region_top)),             # Top-right
    (int(middle_x), int(region_top))       # Top-left
]

# Initialize video writer
video_writer = cv2.VideoWriter("/content/drive/My Drive/YOLO/Datasets/OutputNum2MiniProjectDivide5555555555.mp4",
                               cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

# Initialize counters for left and right regions
left_count = 0
right_count = 0

# Custom function to draw larger labels
def draw_large_label(img, label, coords, color):
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1.2
    thickness = 2
    text_size = cv2.getTextSize(label, font, font_scale, thickness)[0]

    # Draw background rectangle
    cv2.rectangle(img, (int(coords[0]), int(coords[1]) - text_size[1] - 10),
                  (int(coords[0]) + text_size[0], int(coords[1])), color, -1)

    # Draw text
    cv2.putText(img, label, (int(coords[0]), int(coords[1]) - 5), font, font_scale,
                (255, 255, 255), thickness, cv2.LINE_AA)

# Function to draw counting regions
def draw_counting_regions(img):
    cv2.rectangle(img, (0, int(region_top)), (int(w), int(region_bottom)), (0, 255, 0), 2)
    cv2.line(img, (int(middle_x), int(region_top)), (int(middle_x), int(region_bottom)), (0, 255, 0), 2)

# Function to check if a point is within a given region
def is_within_region(x, y, region_points):
    return region_points[0][0] <= x <= region_points[1][0] and region_points[2][1] <= y <= region_points[0][1]

while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break

    # Modify tracking parameters for faster moving objects
    tracks = model.track(im0, persist=True, show=False, iou=0.4, conf=0.3, max_det=50)

    # Draw counting regions
    draw_counting_regions(im0)

    # Draw larger labels for each tracked object, count cars in left and right regions, and draw bounding boxes
    if tracks[0].boxes.id is not None:
        for box, cls, track_id in zip(tracks[0].boxes.xywh, tracks[0].boxes.cls, tracks[0].boxes.id):
            x, y, w, h = box
            label = f'{model.names[int(cls)]} {int(track_id)}'
            draw_large_label(im0, label, (int(x - w/2), int(y - h/2)), (0, 255, 0))

            # Draw bounding box
            top_left = (int(x - w / 2), int(y - h / 2))
            bottom_right = (int(x + w / 2), int(y + h / 2))
            cv2.rectangle(im0, top_left, bottom_right, (0, 255, 0), 2)

            # Determine if the center of the box is in the left or right region
            center_x = int(x)
            center_y = int(y)

            if is_within_region(center_x, center_y, left_region_points):
                left_count += 1
            elif is_within_region(center_x, center_y, right_region_points):
                right_count += 1

    # Display the counts on the frame
    cv2.putText(im0, f'Left Count: {left_count}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.putText(im0, f'Right Count: {right_count}', (int(w) - 300, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    video_writer.write(im0)

cap.release()
video_writer.release()
cv2.destroyAllWindows()



0: 384x640 12 cars, 1 truck, 144.6ms
Speed: 3.3ms preprocess, 144.6ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 cars, 1 truck, 171.2ms
Speed: 3.6ms preprocess, 171.2ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 cars, 1 truck, 162.5ms
Speed: 4.2ms preprocess, 162.5ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 cars, 2 trucks, 166.3ms
Speed: 5.0ms preprocess, 166.3ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 cars, 2 trucks, 156.1ms
Speed: 5.2ms preprocess, 156.1ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 cars, 2 trucks, 159.2ms
Speed: 4.4ms preprocess, 159.2ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 cars, 2 trucks, 170.7ms
Speed: 4.2ms preprocess, 170.7ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 cars, 2 trucks, 158.5ms
Speed

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO, solutions
from collections import defaultdict

# Load the YOLO model
model = YOLO("yolov8n.pt")

# Open the video file
cap = cv2.VideoCapture("/content/drive/My Drive/YOLO/Datasets/Video.mp4")
assert cap.isOpened(), "Error reading video file"

# Get video properties
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

# Define new region points for a taller, vertical region on the left side
# Assuming we want the region to be 1/5 of the video width and 2/3 of the video height
region_width = w // 5
region_height = 2 * h // 3
region_top = (h - region_height) // 2
region_bottom = region_top + region_height

region_points = [
    (0, region_bottom),            # Bottom-left
    (region_width, region_bottom), # Bottom-right
    (region_width, region_top),    # Top-right
    (0, region_top)                # Top-left
]

# Initialize video writer
video_writer = cv2.VideoWriter("/content/drive/My Drive/YOLO/Datasets/OutputMiniProjectWithSpeedAndTrack.mp4",
                               cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

# Initialize Object Counter with modified parameters
counter = solutions.ObjectCounter(
    view_img=True,
    reg_pts=region_points,
    names=model.names,
    draw_tracks=True,
    line_thickness=3  # Increased from 2 to 3 for wider boxes
)

# Custom function to draw larger labels
def draw_large_label(img, label, coords, color):
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1.2
    thickness = 2
    text_size = cv2.getTextSize(label, font, font_scale, thickness)[0]

    # Draw background rectangle
    cv2.rectangle(img, (coords[0], coords[1] - text_size[1] - 10),
                  (coords[0] + text_size[0], coords[1]), color, -1)

    # Draw text
    cv2.putText(img, label, (coords[0], coords[1] - 5), font, font_scale,
                (255, 255, 255), thickness, cv2.LINE_AA)

# Initialize variables for speed calculation
previous_positions = defaultdict(lambda: None)
speeds = defaultdict(lambda: None)
pixels_per_meter = 30  # This needs to be calibrated based on your video

frame_count = 0
while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break

    # Modify tracking parameters for faster moving objects
    tracks = model.track(im0, persist=True, show=False, iou=0.4, conf=0.3, max_det=50)

    im0 = counter.start_counting(im0, tracks)

    # Calculate and draw speeds for each tracked object
    if tracks[0].boxes.id is not None:
        for box, cls, track_id in zip(tracks[0].boxes.xywh, tracks[0].boxes.cls, tracks[0].boxes.id):
            x, y, w, h = box
            center = (int(x), int(y))

            # Calculate speed
            if previous_positions[track_id] is not None:
                distance = np.linalg.norm(np.array(center) - np.array(previous_positions[track_id]))
                speed = distance / (1/fps) / pixels_per_meter * 3.6  # km/h
                speeds[track_id] = speed

            previous_positions[track_id] = center

            # Prepare label
            class_name = model.names[int(cls)]
            speed_text = f"{speeds[track_id]:.1f} km/h" if speeds[track_id] is not None else "N/A"
            label = f'{class_name} {int(track_id)} {speed_text}'

            # Draw label
            draw_large_label(im0, label, (int(x - w/2), int(y - h/2)), (0, 255, 0))

    video_writer.write(im0)
    frame_count += 1

cap.release()
video_writer.release()
cv2.destroyAllWindows()

WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

Polygon Counter Initiated.

0: 640x384 2 motorcycles, 134.2ms
Speed: 3.5ms preprocess, 134.2ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 motorcycles, 161.1ms
Speed: 4.0ms preprocess, 161.1ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 1 motorcycle, 140.4ms
Speed: 4.7ms preprocess, 140.4ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 1 motorcycle, 165.2ms
Speed: 4.2ms preprocess, 165.2ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 182.0ms
Speed: 4.4ms preprocess, 182.0ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 persons, 3 motorcycles, 167.9ms
Speed: 3.9ms preprocess, 167.9ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 1 motorcycle, 138.1ms
Speed: 4.4ms preprocess, 138.1ms 

In [ ]:
video = cv2.VideoCapture('/content/drive/My Drive/YOLO/Datasets/Video.mp4')

## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [ ]:
output = cv2.VideoWriter('/content/drive/My Drive/YOLO/Datasets/YOLO_Output.mp4',
                         cv2.VideoWriter_fourcc(*'MP4V'),
                         int(video.get(cv2.CAP_PROP_FPS)),
                         (int(video.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))))

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

## Save and Submit Your Work

Submit both the notebook and the output video